In [1]:
from langchain_community.document_loaders import DirectoryLoader, TextLoader
from langchain_openai import OpenAIEmbeddings
from langchain_core.documents import Document
from pathlib import Path
import re

In [2]:
# 법령을 각 조로 분리해서 document화

p = Path('./data')
pattern =r'(?=제\s*\d+\s*조(?:의\s*\d+)?\s*\([^)]+\))'
documents=[]
for file_path in p.glob('*.txt'):
    file_name = str(file_path).split('/')[-1].replace('.txt', '')
    with open(file_path, 'r', encoding='utf-8') as f:
        text = f.read()
    articles = re.split(pattern, text)
    articles = [c.strip() for c in articles if c.strip()]

    for i, article in enumerate(articles):
        doc = Document(
            page_content=article, metadata={'source': str(file_path), 'title': file_name, 'seq': i + 1}
        )
        documents.append(doc)

In [ ]:
# 법령으로 구분된 파일을 불러와서 법령 통째로 document화

leader = DirectoryLoader(
    './data',
    glob='*.txt',
    loader_cls=TextLoader,
    show_progress=True,
)

documents = leader.load()

for doc in documents:
    file_name = doc.metadata['source'].split('/')[-1].replace('.txt', '')
    doc.metadata['title'] = file_name

documents[0].metadata

100%|██████████| 622/622 [00:00<00:00, 3139.71it/s]


{'source': 'data/특별사법경찰관리에 대한 검사의 수사지휘 및 특별사법경찰관리의 수사준칙에 관한 규칙_5.txt',
 'title': '특별사법경찰관리에 대한 검사의 수사지휘 및 특별사법경찰관리의 수사준칙에 관한 규칙_5'}

In [3]:
from stores import init_vector_store, QDRANT_URL

vector_store = init_vector_store(documents)

/Users/tera/Desktop/workspace/skn_21/project/project_03_1team_repo/c_team/stores.py:19: UserWarning: Api key is used with an insecure connection.
  return QdrantClient(
/Users/tera/Desktop/workspace/skn_21/project/project_03_1team_repo/.venv/lib/python3.13/site-packages/langchain_qdrant/qdrant.py:921: UserWarning: Api key is used with an insecure connection.
  client = QdrantClient(**client_options)


In [6]:
from stores import get_client, COLLECTION_NAME
from qdrant_client.models import PayloadSchemaType

client = get_client()

client.create_payload_index(
    collection_name=COLLECTION_NAME,
    field_name="title",
    field_schema=PayloadSchemaType.TEXT,
)

/Users/tera/Desktop/workspace/skn_21/project/project_03_1team_repo/c_team/stores.py:19: UserWarning: Api key is used with an insecure connection.
  return QdrantClient(


UpdateResult(operation_id=105, status=<UpdateStatus.COMPLETED: 'completed'>)